In [120]:
# Import Libraries

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle

In [121]:
# Import Models

fridge_model = pickle.load(open("./models/IoT_Fridge.h5", 'rb'))
garage_door_model = pickle.load(open("./models/IoT_Garage_Door.h5", 'rb'))
gps_tracker_model = pickle.load(open("./models/IoT_GPS_Tracker.h5", 'rb'))
modbus_model = pickle.load(open("./models/IoT_Modbus.h5", 'rb'))
motion_light_model = pickle.load(open("./models/IoT_Motion_Light.h5", 'rb'))
thermostat_model = pickle.load(open("./models/IoT_Thermostat.h5", 'rb'))
weather_model = pickle.load(open("./models/IoT_Weather.h5", 'rb'))


C:\Users\spars\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\spars\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\spars\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.

In [122]:
# Reading Data

data = pd.read_csv("./data/merged_data.csv", low_memory=False)

# dropping NaN or NA values
data = data.dropna()


In [123]:
data.columns

Index(['date-time', 'fridge_temperature', 'fridge_temp_condition',
       'fridge_label', 'fridge_type', 'door_state', 'door_sphone_signal',
       'door_label', 'door_type', 'gps_latitude', 'gps_longitude', 'gps_label',
       'gps_type', 'modbus_FC1_Read_Input_Register',
       'modbus_FC2_Read_Discrete_Value', 'modbus_FC3_Read_Holding_Register',
       'modbus_FC4_Read_Coil', 'modbus_label', 'modbus_type',
       'ml_motion_status', 'ml_light_status', 'ml_label', 'ml_type',
       'thermostat_current_temperature', 'thermostat_status',
       'thermostat_label', 'thermostat_type', 'weather_temperature',
       'weather_pressure', 'weather_humidity', 'weather_label',
       'weather_type'],
      dtype='object')

In [124]:
# encoding string type variables to numeric type (required for model training)

encoder=LabelEncoder()
data['fridge_temperature']=encoder.fit_transform(data['fridge_temperature'])
data['fridge_temp_condition']=encoder.fit_transform(data['fridge_temp_condition'])

data['ml_motion_status']=encoder.fit_transform(data['ml_motion_status'])
data['ml_light_status']=encoder.fit_transform(data['ml_light_status'])
data['ml_label']=encoder.fit_transform(data['ml_label'])


# 'door_state', 'door_sphone_signal', 'door_label', 'door_type'
data['door_type']=encoder.fit_transform(data['door_type'])
data['door_state']=encoder.fit_transform(data['door_state'])
data['door_sphone_signal']=encoder.fit_transform(data['door_sphone_signal'])

data['gps_latitude']=encoder.fit_transform(data['gps_latitude'])
data['gps_longitude']=encoder.fit_transform(data['gps_longitude'])

data['modbus_FC1_Read_Input_Register']=encoder.fit_transform(data['modbus_FC1_Read_Input_Register'])
data['modbus_FC2_Read_Discrete_Value']=encoder.fit_transform(data['modbus_FC2_Read_Discrete_Value'])
data['modbus_FC3_Read_Holding_Register']=encoder.fit_transform(data['modbus_FC3_Read_Holding_Register'])
data['modbus_FC4_Read_Coil']=encoder.fit_transform(data['modbus_FC4_Read_Coil'])

data['thermostat_current_temperature']=encoder.fit_transform(data['thermostat_current_temperature'])
data['thermostat_status']=encoder.fit_transform(data['thermostat_status'])

data['weather_temperature']=encoder.fit_transform(data['weather_temperature'])
data['weather_pressure']=encoder.fit_transform(data['weather_pressure'])
data['weather_humidity']=encoder.fit_transform(data['weather_humidity'])


In [125]:
# Creating different test datasets

x_fridge = data.filter(['fridge_temperature', 'fridge_temp_condition'], axis=1)
x_garage_door = data.filter(['door_state', 'door_sphone_signal'], axis=1)
x_gps_tracker = data.filter(['gps_latitude', 'gps_longitude'], axis=1)
x_modbus = data.filter(['modbus_FC1_Read_Input_Register', 'modbus_FC2_Read_Discrete_Value', 'modbus_FC3_Read_Holding_Register', 'modbus_FC4_Read_Coil'], axis=1)
x_motion_light = data.filter(['ml_motion_status', 'ml_light_status'], axis=1)
x_thermostat = data.filter(['thermostat_current_temperature', 'thermostat_status'], axis=1)
x_weather = data.filter(['weather_temperature', 'weather_pressure', 'weather_humidity'], axis=1)

y_fridge = data['fridge_type']
y_garage_door = data['door_type']
y_gps_tracker = data['gps_type']
y_modbus = data['modbus_type']
y_motion_light = data['ml_type']
y_thermostat = data['thermostat_type']
y_weather = data['weather_type']


In [126]:
# Change Column Names

x_fridge.rename(columns= { 'fridge_temp_condition':'temp_condition'}, inplace=True)
# x_fridge.rename(columns= {'fridge_temperature':'temperature', 'fridge_temp_condition':'temp_condition'}, inplace=True)
x_garage_door.rename(columns={'door_state': 'state', 'door_sphone_signal': 'sphone_signal'}, inplace=True)
x_gps_tracker.rename(columns={'gps_latitude': 'latitude', 'gps_longitude':'longitude'}, inplace=True)
x_modbus.rename(columns={'modbus_FC1_Read_Input_Register':'FC1_Read_Input_Register', 'modbus_FC2_Read_Discrete_Value':'FC2_Read_Discrete_Value', 'modbus_FC3_Read_Holding_Register':'FC3_Read_Holding_Register', 'modbus_FC4_Read_Coil':'FC4_Read_Coil'}, inplace=True)
x_motion_light.rename(columns={'ml_motion_status':'motion_statu', 'ml_light_status':'light_status'}, inplace=True)
# x_motion_light.rename(columns={'ml_motion_status':'motion_status', 'ml_light_status':'light_status'}, inplace=True)
x_thermostat.rename(columns={'thermostat_current_temperature': 'current_temperature'}, inplace=True)
# x_thermostat.rename(columns={'thermostat_current_temperature': 'current_temperature', 'thermostat_status':'status'}, inplace=True)
x_weather.rename(columns={'weather_temperature': 'temperature', 'weather_pressure': 'pressure', 'weather_humidity': 'humidity'}, inplace=True)

y_fridge = y_fridge.rename("type" )
y_garage_door = y_garage_door.rename("type")
y_gps_tracker = y_gps_tracker.rename("type")
y_modbus = y_modbus.rename("type")
y_motion_light = y_motion_light.rename("type")
y_thermostat = y_thermostat.rename("type")
y_weather = y_weather.rename("type")

In [127]:
print(y_fridge.unique())
print(y_garage_door.unique())
print(y_gps_tracker.unique())
print(y_modbus.unique())
print(y_motion_light.unique())
print(y_thermostat.unique())
print(y_weather.unique())

['normal' 'injection' 'password' 'xss' 'backdoor']
[2 1 3 4 0]
['normal' 'injection' 'password' 'xss' 'backdoor']
['normal' 'injection' 'password' 'xss' 'backdoor']
['normal' 'injection' 'password' 'xss' 'backdoor']
['normal' 'injection' 'password' 'xss' 'backdoor']
['normal' 'injection' 'password' 'xss' 'backdoor']


In [128]:
# Splitting Data

x_fridge_train,x_fridge_test,y_fridge_train,y_fridge_test = train_test_split(x_fridge, y_fridge, test_size=0.2)
x_garage_door_train,x_garage_door_test,y_garage_door_train,y_garage_door_test = train_test_split(x_garage_door, y_garage_door, test_size=0.2)
x_gps_tracker_train,x_gps_tracker_test,y_gps_tracker_train,y_gps_tracker_test = train_test_split(x_gps_tracker, y_gps_tracker, test_size=0.2)
x_modbus_train,x_modbus_test,y_modbus_train,y_modbus_test = train_test_split(x_modbus, y_modbus, test_size=0.2)
x_motion_light_train,x_motion_light_test,y_motion_light_train,y_motion_light_test = train_test_split(x_motion_light, y_motion_light, test_size=0.2)
x_thermostat_train,x_thermostat_test,y_thermostat_train,y_thermostat_test = train_test_split(x_thermostat, y_thermostat, test_size=0.2)
x_weather_train,x_weather_test,y_weather_train,y_weather_test = train_test_split(x_weather, y_weather, test_size=0.2)


In [129]:
# Predicting Resuts using models

result_fridge = fridge_model.predict(x_fridge_test)
result_garage_door = garage_door_model.predict(x_garage_door_test)
result_gps_tracker = gps_tracker_model.predict(x_gps_tracker_test)
result_modbus = modbus_model.predict(x_modbus_test)
result_motion_light = motion_light_model.predict(x_motion_light_test)
result_thermostat = thermostat_model.predict(x_thermostat_test)
result_weather = weather_model.predict(x_weather_test)

In [130]:
# Calculating Stats for Bayesian Prediction

attack_dict = {
    'normal': 2,
    'injection': 1,
    'password': 3,
    'xss': 4,
    'backdoor': 0
}
# fridge-0 ,garage_door-1 ,gps_tracker-2 ,modbus-3 ,motion_light-4 ,thermostat-5 ,weather-6

size_attack_total = [0, 0, 0, 0, 0]
size_normal_by_dataset = [0, 0, 0, 0, 0, 0, 0]
size_injection_by_dataset = [0, 0, 0, 0, 0, 0, 0]
size_password_by_dataset = [0, 0, 0, 0, 0, 0, 0]
size_xss_by_dataset = [0, 0, 0, 0, 0, 0, 0]
size_backdoor_by_dataset = [0, 0, 0, 0, 0, 0, 0]

prob_normal_by_dataset = [0, 0, 0, 0, 0, 0, 0]
prob_injection_by_dataset = [0, 0, 0, 0, 0, 0, 0]
prob_password_by_dataset = [0, 0, 0, 0, 0, 0, 0]
prob_xss_by_dataset = [0, 0, 0, 0, 0, 0, 0]
prob_backdoor_by_dataset = [0, 0, 0, 0, 0, 0, 0]

prediction_result_length = min(len(result_fridge), len(result_garage_door), len(result_gps_tracker), len(result_modbus), len(result_motion_light), len(result_thermostat), len(result_weather))


In [131]:
counter = -1
for j in [result_fridge, result_garage_door, result_gps_tracker, result_modbus, result_motion_light, result_thermostat, result_weather]:
    counter = counter + 1
    for i in j:
        if(i == attack_dict['normal']):
            size_normal_by_dataset[counter] = size_normal_by_dataset[counter] + 1
            size_attack_total[attack_dict['normal']] = size_attack_total[attack_dict['normal']] + 1
        elif (i == attack_dict['injection']): 
            size_injection_by_dataset[counter] = size_injection_by_dataset[counter] + 1
            size_attack_total[attack_dict['injection']] = size_attack_total[attack_dict['injection']] + 1
        elif (i == attack_dict['password']):
            size_password_by_dataset[counter] = size_password_by_dataset[counter] + 1
            size_attack_total[attack_dict['password']] = size_attack_total[attack_dict['password']] + 1
        elif (i == attack_dict['xss']):
            size_xss_by_dataset[counter] = size_xss_by_dataset[counter] + 1
            size_attack_total[attack_dict['xss']] = size_attack_total[attack_dict['xss']] + 1
        elif (i == attack_dict['backdoor']):
            size_backdoor_by_dataset[counter] = size_backdoor_by_dataset[counter] + 1
            size_attack_total[attack_dict['backdoor']] = size_attack_total[attack_dict['backdoor']] + 1


In [132]:
# Increase every size by 1 to avoid 0/0 divison

size_attack_total = [x+1 for x in size_attack_total]
size_normal_by_dataset = [x+1 for x in size_normal_by_dataset]
size_injection_by_dataset = [x+1 for x in size_injection_by_dataset]
size_password_by_dataset = [x+1 for x in size_password_by_dataset]
size_xss_by_dataset = [x+1 for x in size_xss_by_dataset]
size_backdoor_by_dataset = [x+1 for x in size_backdoor_by_dataset]

In [133]:
prob_normal_by_dataset = [i / size_attack_total[attack_dict['normal']] for i in size_normal_by_dataset]
prob_injection_by_dataset = [i / size_attack_total[attack_dict['injection']] for i in size_injection_by_dataset]
prob_password_by_dataset = [i / size_attack_total[attack_dict['password']] for i in size_password_by_dataset]
prob_xss_by_dataset = [i / size_attack_total[attack_dict['xss']] for i in size_xss_by_dataset]
prob_backdoor_by_dataset = [i / size_attack_total[attack_dict['backdoor']] for i in size_backdoor_by_dataset]

In [134]:
prob_normal = size_attack_total[attack_dict['normal']] / sum(size_attack_total)
prob_injection = size_attack_total[attack_dict['injection']] / sum(size_attack_total)
prob_password = size_attack_total[attack_dict['password']] / sum(size_attack_total)
prob_xss = size_attack_total[attack_dict['xss']] / sum(size_attack_total)
prob_backdoor = size_attack_total[attack_dict['backdoor']] / sum(size_attack_total)


In [137]:
# Predicting probaility using Bayesian Prediction

result_bayesian = [-1] * prediction_result_length

for i in range(prediction_result_length):
    temp_res = [result_fridge[i], result_garage_door[i], result_gps_tracker[i], result_modbus[i], result_motion_light[i], result_thermostat[i], result_weather[i]]

    normal_prob = len(list(filter(lambda x: x == attack_dict['normal'], temp_res))) / len(temp_res)
    injection_prob = len(list(filter(lambda x: x == attack_dict['injection'], temp_res))) / len(temp_res)
    password_prob = len(list(filter(lambda x: x == attack_dict['password'], temp_res))) / len(temp_res)
    xss_prob = len(list(filter(lambda x: x == attack_dict['xss'], temp_res))) / len(temp_res)
    backdoor_prob = len(list(filter(lambda x: x == attack_dict['backdoor'], temp_res))) / len(temp_res)

    normal_prob = normal_prob * np.prod(prob_normal_by_dataset)
    injection_prob = injection_prob * np.prod(prob_injection_by_dataset)
    password_prob = password_prob * np.prod(prob_password_by_dataset)
    xss_prob = xss_prob * np.prod(prob_xss_by_dataset)
    backdoor_prob = backdoor_prob * np.prod(prob_backdoor_by_dataset)

    final_prob = max(normal_prob, injection_prob, password_prob, xss_prob, backdoor_prob)

    if final_prob == normal_prob:
        result_bayesian[i] = attack_dict['normal']
    elif final_prob == injection_prob:
        result_bayesian[i] = attack_dict['injection']
    elif final_prob == password_prob:
        result_bayesian[i] = attack_dict['password']
    elif final_prob == xss_prob:
        result_bayesian[i] = attack_dict['xss']
    elif final_prob == backdoor_prob:
        result_bayesian[i] = attack_dict['backdoor']
    

In [138]:
result_bayesian

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [141]:
attack_dict_keys = list(attack_dict.keys())
attack_dict_values = list(attack_dict.values())

result_bayesian_named = [attack_dict_keys[attack_dict_values.index(x)] for x in result_bayesian]

In [142]:
result_bayesian_named

['normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',

In [150]:
print(list(set(result_fridge)))
print(list(set(result_garage_door)))
print(list(set(result_gps_tracker)))
print(list(set(result_modbus)))
print(list(set(result_motion_light)))
print(list(set(result_thermostat)))
print(list(set(result_weather)))

['6', '3']
['0']
['6', '7', '5']
['2', '1']
['7']
['4']
['7']
